# Cavern storage capacity for variable heights

In [137]:
import os

import cartopy.crs as ccrs
import contextily as cx
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
from matplotlib.lines import Line2D
from matplotlib_scalebar.scalebar import ScaleBar

from h2ss import capacity as cap
from h2ss import compare
from h2ss import data as rd
from h2ss import functions as fns

import xyzservices.providers as xyz
from bokeh.io import output_notebook
from bokeh.models import EqHistColorMapper, GeoJSONDataSource, HoverTool, Metric, ScaleBar
from bokeh.palettes import inferno
from bokeh.plotting import figure, show

In [15]:
import importlib

In [2]:
output_notebook()

Loading BokehJS ...

In [17]:
ds, extent, exclusions = compare.load_all_data()

In [4]:
caverns = compare.capacity_function(ds=ds, extent=extent, exclusions=exclusions, cavern_diameter=85, cavern_height=120)

In [ ]:
shape = rd.halite_shape(dat_xr=ds).buffer(1000).buffer(-1000)

## Map

In [80]:
# # caverns_wm = gpd.GeoDataFrame(caverns.drop(columns=["geometry"]), geometry=caverns.centroid, crs=caverns.crs)
# # caverns_wm = caverns_wm.to_crs(3857)
# caverns_wm = caverns.to_crs(3857)
# # caverns_wm["cavern_height"] = caverns_wm["cavern_height"].astype(int).astype(str)
# caverns_wm["geometry"] = caverns_wm["geometry"].centroid

In [81]:
# const = list(caverns_wm["cavern_height"].unique())
# palette = inferno(len(const))
# color_map = CategoricalColorMapper(factors=const, palette=palette)

In [106]:
color_map = EqHistColorMapper(palette=tuple(reversed(inferno(6))))

In [172]:
caverns_src = caverns.copy()
# caverns_src["geometry"] = caverns_src.buffer(100)
exclusions["shipwrecks"]["VESSELNAME"].fillna("None", inplace=True)

/tmp/ipykernel_88193/2485813416.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  exclusions["shipwrecks"]["VESSELNAME"].fillna("None", inplace=True)


In [173]:
caverns_src = GeoJSONDataSource(geojson=caverns_src.to_crs(3857).to_json())
shape_src = GeoJSONDataSource(geojson=shape.to_crs(3857).to_json())
wind_farms_src = GeoJSONDataSource(geojson=exclusions["wind_farms"].to_crs(3857).to_json())
shipwrecks_src = GeoJSONDataSource(geojson=exclusions["shipwrecks"].to_crs(3857).to_json())

In [175]:
p = figure(
    title="Kish Basin caverns",
    tools="wheel_zoom, pan, reset, save",
    match_aspect=True,
    x_axis_type="mercator",
    y_axis_type="mercator",
    height=800, width=800
)

p.patches("xs", "ys", source=shape_src, line_color="darkslategrey", line_width=0.5, fill_color="none", legend_label="Kish Basin boundary")
wind_farms_plot = p.patches("xs", "ys", source=wind_farms_src, hatch_color="royalblue", hatch_pattern="left_diagonal_line", fill_color="none", legend_label="Wind farm")
caverns_plot = p.patches(
    "xs",
    "ys",
    source=caverns_src,
    line_width=0.5,
    line_color="white",
    fill_color={"field": "capacity", "transform": color_map},
)
shipwrecks_plot = p.scatter("x", "y", source=shipwrecks_src, marker="x", size=5, color="black", legend_label="Shipwreck")

hover = HoverTool(renderers=[caverns_plot], tooltips=[
    # ("Cavern height [m]", "@cavern_height"),
    ("Type", "Cavern"),
    ("Capacity [GWh]", "@capacity"),
])
p.add_tools(hover)
hover = HoverTool(renderers=[wind_farms_plot], tooltips=[("Type", "Wind farm"), ("Name", "@name")])
p.add_tools(hover)
hover = HoverTool(renderers=[shipwrecks_plot], tooltips=[
    ("Type", "Shipwreck"),
    ("Name", "@VESSELNAME"),
    ("Vessel type", "@VESSELTYPE"),
])
p.add_tools(hover)

p.legend.location = "bottom_right"
p.legend.click_policy="hide"
color_bar = caverns_plot.construct_color_bar(title="Cavern capacity [GWh]")
p.add_layout(color_bar)
p.grid.grid_line_color = "lightgrey"
p.add_tile(xyz.CartoDB.Voyager)
# scale_bar = ScaleBar(
#     # range=p.x_range,
#     location="bottom_left",
#     # unit="m",
# )
# p.add_layout(scale_bar)

show(p)